#### AHSI - Azure Hyperscale for Seismic Imaging
Epic 5.	AHSI ALICIA (Azure hyperscaLe seIsmiC Imaging bAtch) 

Reproducible guide based on these [instructions](https://github.com/slimgroup/Azure2019)

##### 1. Use Azure portal to create a resource group and these 2 resources under it:
 - control_plane_ACR, for example [created via Azure Portal](https://docs.microsoft.com/en-us/azure/container-registry/container-registry-get-started-portal). Use portal "Access Keys" blade to note:
   - Registry name, eg controlplaneacr
   - Login server, e.g. controlplaneacr.azurecr.io
   - Username
   - password   
   These info bits will be used in `010_create_control_plane_docker_image.ipynb` below.  
   
 - an [Azure Ubuntu DSVM](https://docs.microsoft.com/en-us/azure/machine-learning/data-science-virtual-machine/dsvm-ubuntu-intro):  
   - open  two ports (named below JupPort1 and JupPort2, .e.g. 9998 and 9999) for Jupyter. See these [instructions](https://github.com/microsoft/AMLSDKRModelsOperationalization#vm-provisioning-and-configuration-via-azure-cli) for details.  
   __*Security Note*__: Secure [access](https://docs.microsoft.com/en-us/azure/marketplace/cloud-partner-portal/virtual-machine/cpp-connect-vm#connect-to-a-linux-based-vm) to VMs and to Jupyter notebook servers is paramount, but outside the scope of this tutorial. We use here simple security measures like ssh via login and password using non-standard ports. More secure ways use [ssh keys](https://docs.microsoft.com/en-us/azure/virtual-machines/linux/mac-create-ssh-keys) and [jit](https://docs.microsoft.com/en-us/azure/security-center/security-center-just-in-time). It is highly recommended to address the access security issue before starting an Azure development project.  
     
   - clone this repo (e.g. in /datadrive01/prj/Azure2019)
   - [have](https://docs.docker.com/get-started/) docker [installed](https://docs.docker.com/engine/install/ubuntu/). 
     - Method 2.a has __no__ other dependencies.  
     - Method 2.b requires [conda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/) 
     
  
   
##### 2. Create running env in either of the two following ways:

   2.a Create a conda env "on the fly" in a miniconda3 docker image/container. This is the recommended, most repro option.    
   First, create file __notebooks/not_shared/base.env__ with this content:
```
vm_Jupyter_port=8888
sibling_docker_env_file=not_shared/sibling_docker.env
```
   Change as needed:
    - __vm_Jupyter_port__ is the Jupyter port opened on the VM as mentioned above
    - local directory not_shared for sibling_docker_env_file will be vreated my Makefile as described below. It points to the local host directory that will be mounted inside running env docker container as /workspace  
     
   Then start Jupyter Notebook server in the running env docker ontainer, either via tmux:    
```bash
tmux new -s miniconda3_02 'make jupyter-base-env' 
```
   or simply:
```bash
make jupyter-base-env

```  

   Jupyter Server started above should then be available at vm_Jupyter_port, e.g.:   
   http://[AZURE_DSVM].[AZURE_DSVM_REGION].cloudapp.azure.com:8888/notebooks/notebooks/000_Instructions.ipynb

   2.b (Alternative option) Create a conda env based on `azure_devito_conda_dep_file_control_plane.yml` file.

  
   
##### 3. Run `notebooks/010_create_control_plane_docker_image.ipynb` 
Follow the instructions (editing and Azure sign-in steps required) in  and run it to create the control plane and daks docker images.  
  
     
   
__________________________________________________________   
__________________________________________________________   
__________________________________________________________   
   
    
##### 5. Run the azure_batch_shipyard_devito docker image created above in container
Use command below. 
  Change as needed:
  - the port forwarding __-p JupPort2:8888__, e.g. -p 9999:8888.  
  - use daks_base_docker_image that has been created in  `020_deployK8sClusterAndDask.ipynb`.  
  - use daks_ssh_dir with ssh keys created in `020_deployK8sClusterAndDask.ipynb`
  - use the EXTERNAL-IP of the devito-server (LoadBalancer) to run devito in dask in Azure as described below. 
    
    
```
docker run \
 -it \
 --rm  \
 --name azure_batch_shipyard_devito_container01 \
 -v $(pwd):/workspace:rw \
 -v /datadrive01/prj/Azure2019/notebooks/./not_shared/azuredevitogatechssh:/root/.ssh:ro \
 -v /usr/bin/docker:/usr/bin/docker \
 -v /var/run/docker.sock:/var/run/docker.sock \
 fwi01acr.azurecr.io/azure_batch_shipyard_devito \
 /bin/bash
 
```

  
   
##### 7. Start azure batch shipyard compute pool and run jobs
root@491f894856fe:/# cd /workspace/src/AzureBatch/shipyard/config_msft_vnettest/
root@491f894856fe:/workspace/src/AzureBatch/shipyard/config_msft_vnettest#


root@f57986742e96:/workspace/src/AzureBatch/shipyard/config_gcc# python3 /opt/batch-shipyard/shipyard.py pool add -v&  
root@f57986742e96:/workspace/src/AzureBatch/shipyard/config_gcc# python3 /opt/batch-shipyard/shipyard.py jobs add --tail stdout.txt -v&


In [2]:
!jupyter nbconvert 000_Instructions.ipynb --to html

[NbConvertApp] Converting notebook 000_Instructions.ipynb to html
[NbConvertApp] Writing 281956 bytes to 000_Instructions.html


In [3]:
!jupyter nbconvert 000_Instructions.ipynb --to markdown 

[NbConvertApp] Converting notebook 000_Instructions.ipynb to markdown
[NbConvertApp] Writing 8055 bytes to 000_Instructions.md
